# Meilisearch

> [Meilisearch](https://meilisearch.com) is an open-source, lightning-fast, and hyper relevant search engine. It comes with great defaults to help developers build snappy search experiences. 
>
> You can [self-host Meilisearch](https://www.meilisearch.com/docs/learn/getting_started/installation#local-installation) or run on [Meilisearch Cloud](https://www.meilisearch.com/pricing).

Meilisearch v1.3 supports vector search. This page guides you through integrating Meilisearch as a vector store and using it to perform vector search.

## Setup

### Launching a Meilisearch instance

You will need a running Meilisearch instance to use as your vector store. You can run [Meilisearch in local](https://www.meilisearch.com/docs/learn/getting_started/installation#local-installation) or create a [Meilisearch Cloud](https://cloud.meilisearch.com/) account.

As of Meilisearch v1.3, vector storage is an experimental feature. After launching your Meilisearch instance, you need to **enable vector storage**. For self-hosted Meilisearch, read the docs on [enabling experimental features](https://www.meilisearch.com/docs/learn/experimental/vector-search). On **Meilisearch Cloud**, enable _Vector Store_ via your project _Settings_ page.

You should now have a running Meilisearch instance with vector storage enabled. 🎉

### Credentials

To interact with your Meilisearch instance, the Meilisearch SDK needs a host (URL of your instance) and an API key.

**Host**

- In **local**, the default host is `localhost:7700`
- On **Meilisearch Cloud**, find the host in your project _Settings_ page

**API keys**

Meilisearch instance provides you with three API keys out of the box: 
- A `MASTER KEY` — it should only be used to create your Meilisearch instance
- A `ADMIN KEY` — use it only server-side to update your database and its settings
- A `SEARCH KEY` — a key that you can safely share in front-end applications

You can create [additional API keys](https://www.meilisearch.com/docs/learn/security/master_api_keys) as needed.

## Installation

This guide uses the [Meilisearch Python SDK](https://github.com/meilisearch/meilisearch-python).

You can install it by running:

In [ ]:
!pip install meilisearch


To learn more about Meilisearch Python, refer to the in-depth
Meilisearch Python documentation: https://meilisearch.github.io/meilisearch-python/.

### Provide OpenAI access key
We want to use `OpenAIEmbeddings` so we have to get the OpenAI API Key.

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

If you don't want to have to send your client or Meilisearch instance address, you can directly configure the [environment variables](https://www.meilisearch.com/docs/learn/configuration/instance_options#environment-variables).

In [ ]:
os.environ["MEILI_HTTP_ADDR"] = getpass.getpass("Meilisearch HTTP address and port:")

In [ ]:
os.environ["MEILI_MASTER_KEY"] = getpass.getpass("Meilisearch API Key:")

## Add texts with user-friendly interface

In [5]:
from langchain.vectorstores import Meilisearch
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

embeddings = OpenAIEmbeddings()

In [6]:
with open("../../../state_of_the_union.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

There are several ways to instantiate the connection with Meilisearch, you can either send your `client` or just the `url` of your instance. `api_key` is optional, send it only if your instance needs a key.

In [ ]:
docsearch = Meilisearch.from_texts(texts=texts, embedding=embeddings)

## Add documents with user-friendly interface

In [4]:
from langchain.document_loaders import TextLoader

loader = TextLoader("../../../state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [5]:
docsearch = Meilisearch.from_documents(documents=documents, embedding=embeddings)

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

## Add documents by creating Vectorstore

In [ ]:
from langchain.vectorstores import Meilisearch
import meilisearch

client = meilisearch.Client(url="http://127.0.0.1:7700", api_key="***")
vectorstore = Meilisearch(
    embedding=embeddings, client=client, index_name="langchain_demo", text_key="text"
)
vectorstore.add_documents(documents)

## Similarity Search with score

In [ ]:
docs_and_scores = docsearch.similarity_search_with_score(query)
docs_and_scores[0]

## Similarity Search by vector

In [ ]:
embedding_vector = embeddings.embed_query(query)
docs_and_scores = docsearch.similarity_search_by_vector(embedding_vector)
docs_and_scores[0]

## Additional resources

- [Meilisearch repository](https://github.com/meilisearch/meilisearch)
- [Meilisearch Python SDK](https://github.com/meilisearch/meilisearch-python)
- [Meilisearch documentation](https://www.meilisearch.com/docs/)